# Getting Started

This notebook is a tutorial for how to load and run `traffic_prophet`, based on . It assumes you've already installed the Python package dependencies listed in `requirements.txt`, and created (or refreshed, if necessary) the tables

```
prj_volume.tp_centreline_lonlat.sql
prj_volume.tp_centreline_volumes.sql
prj_volume.tp_daily_volumes.sql
```

The creation scripts for these can be found in `input_data/flow`.

## Configure `traffic_prophet`

To run `traffic_prophet`, you need to set up a `config.py` file in the `../../traffic_prophet/` directory. Also in that directory is a `config_template.py` file that shows how to create `config.py`. For a default configuration, simply copy the contents of `config_template.py` into a new `config.py` file.

## Import Packages

In [1]:
import sys
import pathlib
import configparser
import numpy as np
import pandas as pd
# The package's location relative to this Jupyter Notebook.
sys.path.append('../../')

Load your Postgres credentials using `ConfigParser`.

In [2]:
config = configparser.ConfigParser()
config.read({YOUR CONFIG FILE PATH AND NAME});
postgres_settings = config['POSTGRES']

As it stands, Traffic Prophet does not have a single run command. Instead, submodules must be manually imported.

In [3]:
from traffic_prophet import connection
from traffic_prophet.countmatch import reader
from traffic_prophet.countmatch import permcount as pc
from traffic_prophet.countmatch import neighbour as nbr
import traffic_prophet.countmatch.matcher as mt

Import configuration dictionary:

In [4]:
from traffic_prophet import cfg
cfg.cm['verbose'] = True

Create connection objects for volume and lon-lat tables.

In [ ]:
vol_conn = connection.Connection(postgres_settings, 'prj_volume.tp_daily_volumes_clean')
ll_conn = connection.Connection(postgres_settings, 'prj_volume.tp_centreline_lonlat')

## Run CountMatch

### Read in Data

In [ ]:
%%time
tcs = reader.read(vol_conn)

### Separate Short Term Traffic Counts (STTCs) from Permanent Counts (PTCs)

In [ ]:
%%time
pc.get_ptcs_sttcs(tcs)

### Determine Nearest PTC for each STTC

In [ ]:
%%time
ptc_ids = np.unique(np.abs(list(tcs.ptcs.keys())))
nb = nbr.NeighbourLonLatEuclidean(ll_conn, 20, ptc_ids)
nb.find_neighbours()

### Run Matching Algorithm

There are two matcher classes - `MatcherStandard`, which is based on TEPS, and `MatcherBagheri`, which is based on Bagheri et al. 2014's original formulation. `mb = mt.Matcher('Standard', tcs, nb)` creates a standard TEPS matcher; use `mb = mt.Matcher('Bagheri', tcs, nb)` to create a Bagheri-based one.

In [ ]:
mb = mt.Matcher('Standard', tcs, nb)

In [ ]:
%%time
results = mb.estimate_aadts(2016)

`results[0]` are the STTC AADTs; `results[1]` are the PTC AADTs.

## Visualize Results

To visualize the results, you'll need to install

```
geopandas>=0.7.0
matplotlib>=3.2.2
```

to read in geometries from the Postgres database.

In [ ]:
import psycopg2
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as colors

%matplotlib inline

In [ ]:
# Average results across all directions of travel to get a AADT on (directionless) centreline dataset.
results_nodir = pd.concat([results[0], results[1]])
results_nodir['centreline_id'] = np.abs(results_nodir['Count ID'])
results_nodir = pd.DataFrame(
    results_nodir.groupby('centreline_id')['AADT Estimate'].mean())

In [ ]:
sql_query = """SELECT geo_id,
       geom
FROM gis.centreline;"""

with psycopg2.connect(**vol_conn.credentials) as db_con:
    centreline_segments = gpd.read_postgis(sql_query, db_con)

In [ ]:
centreline_segments = pd.merge(
    centreline_segments, results_nodir, left_on='geo_id',
    right_on='centreline_id', how='inner')

centreline_segments.to_crs({'init': 'epsg:3347'}, inplace=True)
centreline_segments['geom'] = centreline_segments.rotate(
    -28., origin=centreline_segments.unary_union.centroid)

In [ ]:
cmap = 'viridis'
vmin = 2e2
vmax = 1e5

fig = plt.figure(figsize=(16, 9))
fig.patch.set_facecolor('#000000')
ax1 = fig.add_axes([0.05, 0.05, 0.8, 0.85])
ax1.axis('off')

centreline_segments.plot(
    column='AADT Estimate', ax=ax1, cmap=cmap,
    norm=colors.LogNorm(vmin=vmin, vmax=vmax))

cax1 = fig.add_axes([0.875, 0.05, 0.03, 0.85])
cb1 = fig.colorbar(
    plt.cm.ScalarMappable(
        cmap=cmap,
        norm=colors.LogNorm(vmin=vmin, vmax=vmax)),
    cax=cax1)
cb1.set_label("AADT", labelpad=10, fontsize=20, color='w')
cax1.tick_params(labelsize=14, labelcolor='w')

At least on a log scale, that seems vaguely reasonable. Note that while TEPS properly handles 400-series highways, **they are currently not available in Traffic Prophet.**